In [1]:
from tqdm import tqdm
from whoosh import scoring
from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT, ID
from whoosh.analysis import StemmingAnalyzer
from whoosh.index import open_dir
from whoosh.qparser import QueryParser, OrGroup, AndGroup
import os
import json
import pandas as pd

In [34]:
def load_hp_synonyms():
    # load hpo from json
    hpo_json = json.load(open('hp.json'))
    synonym_dict_list = []
    nodes = hpo_json['graphs'][0]['nodes']
    for node in nodes:
        # "id" : "http://purl.obolibrary.org/obo/HP_0000016"
        try:
            id_component_list = node['id'].split('/')
            if 'HP_' in id_component_list[-1]:
                synonym_dict = {}
                synonym_dict['hp_id'] = id_component_list[-1]
                synonym_dict['name'] = node['lbl']
                synonym_dict['synonyms'] = [node['lbl']]
                if 'meta' in node:
                    if 'synonyms' in node['meta']:
                        synonyms = node['meta']['synonyms']
                        for synonym in synonyms:
                            synonym_dict['synonyms'].append(synonym['val'])
                synonym_dict_list.append(synonym_dict)
        except Exception as e:
            print(e)
            pass
    return synonym_dict_list

def create_index(index_dir, synonym_dict_list):
    # Create an index and schema
    stem_ana = StemmingAnalyzer()
    custom_schema = Schema(hp_id=ID(stored=True),
                hp_desc=TEXT(stored=True, analyzer=stem_ana),
    )
    # create if not exist
    if not os.path.exists(index_dir):
        os.mkdir(index_dir)
    else:
        # delete forecely and recreate
        import shutil
        shutil.rmtree(index_dir)
        os.mkdir(index_dir)
    index = create_in(index_dir, custom_schema)

    # Open the index
    index = open_dir(index_dir)

    # Create a writer to add documents to the index
    writer = index.writer()

    # Add documents to the index
    for i in tqdm(range(len(synonym_dict_list))):
        phrase = synonym_dict_list[i]
        synonyms = phrase['synonyms']
        # synonyms.extend([phrase['name']])
        for synonym in synonyms:
            writer.add_document(hp_id=str(phrase['hp_id']),
                                hp_desc=synonym
                                )
    writer.commit()
    return 0

def query_terms(index_dir, query_name_list, output, top_k=1):
    def custom_scoring(searcher, fieldname, text, matcher):
        # frequency = scoring.Frequency().scorer(searcher, fieldname, text).score(matcher)
        # tfidf =  scoring.TF_IDF().scorer(searcher, fieldname, text).score(matcher)
        bm25 = scoring.BM25F().scorer(searcher, fieldname, text).score(matcher)
        return bm25
    
    query_results = []
    index = open_dir(index_dir)
    my_weighting = scoring.FunctionWeighting(custom_scoring)
    # scoring.BM25F(B=10, K1=0.1)
    searcher = index.searcher(weighting=scoring.BM25F(B=10, K1=0.1))
    query_parser = QueryParser('hp_desc', schema=index.schema, group=OrGroup)
    # Tokenize the query text using the same analyzer
    for query_name in tqdm(query_name_list):
        query = query_parser.parse('{}'.format(query_name))
        results = searcher.search(query, limit=1, scored=False)
        # Retrieve the top k matching phrase
        if len(results) > 0:
            # Retrieve and print the ranked matching phrases
            # top k only
            results = results[:top_k]
            for i, result in enumerate(results):
                hp_desc = result['hp_desc']
                hp_id = result['hp_id']
                score = result.score
                query_results.append({'query_name': query_name, 'hp_desc': hp_desc, 'hp_id': hp_id, 'score': score})
        else:
            query_results.append({'query_name': query_name, 'hp_desc': '', 'hp_id': '', 'score': -1})
    searcher.close()
    query_results_df = pd.DataFrame(query_results)
    query_results_df.to_csv(output, index=False)   
    return query_results_df

In [10]:
synonym_dict_list = load_hp_synonyms()
index_dir = './woosh_index'
create_index(index_dir, synonym_dict_list)

100%|██████████| 18263/18263 [00:05<00:00, 3596.37it/s]


0

In [21]:
query_name_list = pd.read_csv('synonym_df.csv')['synonym'].tolist()
output = './woosh_query_results.csv'
query_terms(index_dir, query_name_list, output, top_k=1)

  0%|          | 0/498 [00:00<?, ?it/s]

100%|██████████| 498/498 [00:15<00:00, 32.94it/s]


,query_name,hp_desc,hp_id,score
0,Liver fibrosis,Liver fibrosis,HP_0001395,23.713547
1,Hepatic fibrosis,Hepatic fibrosis,HP_0001395,24.282408
2,Portal tract fibrosis,Portal fibrosis,HP_0006580,26.451651
3,Fibrosis of the liver,Liver fibrosis,HP_0001395,23.713547
4,Fibrotic liver disease,Liver disease,HP_0001392,20.715952
...,...,...,...,...
493,Bovine milk hypersensitivity,Hypersensitivity,HP_0041092,24.677188
494,Dairy allergy,Allergy to dairy,HP_0410327,25.211736
495,Allergy to cow's milk,Cow milk allergy,HP_0100327,27.288984
496,Milk protein allergy,Milk allergy,HP_0100327,23.474081


In [3]:
# typos evaluation
query_name_list = pd.read_csv('hp_typos_list.csv')['typo'].tolist()
output = './woosh_typos_query_results.csv'
index_dir = './woosh_index'
query_results = query_terms(index_dir, query_name_list, output, top_k=1)
evaluation_df = query_results.merge(pd.read_csv('hp_typos_list.csv'),left_on='query_name',right_on='typo')
evaluation_df['correct_match'] = evaluation_df['hp_id_x'] == evaluation_df['hp_id_y']
# rate of correct_match
evaluation_df['correct_match'].sum()/evaluation_df['correct_match'].shape[0]

100%|██████████| 31536/31536 [03:20<00:00, 157.25it/s]


0.07302301984907096

In [22]:
# gpt synonyms evaluation
query_name_list = pd.read_csv('gpt_synonym_df.csv')['synonym'].tolist()
output = './woosh_gpt_synonyms_query_results.csv'
index_dir = './woosh_index'
query_results = query_terms(index_dir, query_name_list, output, top_k=1)
evaluation_df = query_results.merge(pd.read_csv('gpt_synonym_df.csv'),left_on='query_name',right_on='synonym')
evaluation_df['correct_match'] = evaluation_df['hp_id_x'] == evaluation_df['hp_id_y']
# rate of correct_match
evaluation_df['correct_match'].sum()/evaluation_df['correct_match'].shape[0]

  0%|          | 0/498 [00:00<?, ?it/s]

100%|██████████| 498/498 [00:15<00:00, 31.74it/s]


0.2891566265060241

In [29]:
# hpo single typo evaluation
import re

input_json = json.load(open('./single_typo.json'))
query_name_id_list = []
for i in input_json:
    # The Human Phenotype Ontology term Multicystic dysplastic kidney is identified by the HPO ID
    # re match
    m1 = re.match('The Human Phenotype Ontology term (.+?) is identified by the HPO ID', i['input'])
    m2 = re.match('The HPO term (.+?) represents', i['input'])
    m3 = re.match('The HPO ID of (.+?) corresponds to', i['input'])
    m4 = re.match('The HPO ID of (.+?) is', i['input'])
    m5 = re.match('The HPO ID of the concept (.+?) is', i['input'])

    if m1 or m2 or m3 or m4:
        if m1 is not None:
            query_name_id_list.append({'query_term':m1.group(1),'hp_id':i['output']})
        if m2 is not None:
            query_name_id_list.append({'query_term':m2.group(1),'hp_id':i['output']})
        if m3 is not None:
            query_name_id_list.append({'query_term':m3.group(1),'hp_id':i['output']})
        if m5 is not None:
            query_name_id_list.append({'query_term':m5.group(1),'hp_id':i['output']})
        else:
            if m4 is not None:
                query_name_id_list.append({'query_term':m4.group(1),'hp_id':i['output']})
    else:
        print(i['input'])

query_name_id_list_df = pd.DataFrame(query_name_id_list)
query_name_id_list_df['hp_id'] = query_name_id_list_df['hp_id'].apply(lambda x: x.replace(':','_'))
query_name_id_list_df = query_name_id_list_df.drop_duplicates()

In [33]:
output = './woosh_hpo_single_tpypo_query_results.csv'
index_dir = './woosh_index'
query_name_list = list(set(query_name_id_list_df['query_term'].tolist()))
query_results = query_terms(index_dir, query_name_list, output, top_k=1)

  0%|          | 0/17938 [00:00<?, ?it/s]

  3%|▎         | 521/17938 [00:19<10:50, 26.76it/s]


KeyboardInterrupt: 

In [20]:
evaluation_df = query_results.merge(query_name_id_list_df,left_on='query_name',right_on='query_term')
evaluation_df['correct_match'] = evaluation_df['hp_id_x'] == evaluation_df['hp_id_y']
# rate of correct_match
evaluation_df['correct_match'].sum()/evaluation_df['correct_match'].shape[0]

0.40480961923847697